In [38]:
import dataclasses
import pandas as pd
# 表示量
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

# データを読み込む
tag_skill_df = pd.read_csv("./Notion/Tags×スキル対応表.csv")
skill_df = pd.read_csv("./iCD2022/スキル一覧（スキル項目）.csv")
knowledge_df = pd.read_csv("./iCD2022/スキル一覧（知識項目）.csv")
knowledge_df = knowledge_df.drop("スキル項目識別子", axis=1)
career_skill_df = pd.read_csv("./iCD2022/職種×スキル対応表.csv")
career_df = pd.read_csv("./iCD2022/職種一覧.csv")


### 1. Tags×スキル対応表の作成
Tagsとスキルの対応関係の表を手動で作成する。
1. スキル一覧（スキル項目）を眺めて対応しそうなスキルを見つける。
2. スキル一覧（知識項目）を眺めてさらに対応しそうな知識項目を見つける。
3. Tags×スキル対応表に書き出す。

<img src='./iCD2022/skill_overview.png' alt="スキルディクショナリ構成図" />

#### 1.1. スキル一覧（スキル項目）を眺めて対応しそうなスキルを見つける。

In [ ]:
@dataclasses.dataclass(frozen=True)
class SkillCategory:
    all: str = '.*'
    methodology: str = 'メソドロジ'
    technology: str = 'テクノロジ'
    human: str = 'ITヒューマンスキル'
    related: str = '関連知識'

@dataclasses.dataclass(frozen=True)
class SkillClass:
    all: str = '.*'
    # メソドロジ
    strategy: str = '.*戦略.*'                     # Skill Code: S11      
    planning: str = '.*企画.*'                     # Skill Code: S12
    implement: str = '.*実装.*'                    # Skill Code: S13
    utilization: str = '.*利活用.*'                # Skill Code: S14
    support: str = '.*支援活動.*'                  # Skill Code: S15
    # テクノロジ
    system: str = '.*（システム）.*'               # Skill Code: S21
    development: str = '.*（開発）.*'              # Skill Code: S22
    operation: str = '.*保守・運用.*'              # Skill Code: S23
    non_function: str = '.*非機能要件.*'           # Skill Code: S24
    embedded: str = '.*組込み・計測・制御.*'       # Skill Code: S25
    common: str = '.*共通技術.*'                  # Skill Code: S26
    # ITヒューマンスキル
    creativity: str = '.*創造力.*'                    # Skill Code: S41001
    action: str = '.*実行・実践力.*'                  # Skill Code: S41002
    communication: str = '.*コミュニケーション力.*'    # Skill Code: S41003
    # 関連知識
    business: str = '.*ビジネスインダストリ.*'           # Skill Code: S31001
    corporate: str = '.*企業活動.*'                     # Skill Code: S31002
    standard: str = '.*法規・基準・標準.*'               # Skill Code: S31003
    
# スキルカテゴリを絞る
# SkillCategory. 以下の部分を補完機能を使用して入力する
# select_df = skill_df.query("スキルカテゴリ.str.match('{0}')".format(SkillCategory.all))

# スキル分類を絞る
# SkillClass. 以下の部分を補完機能を使用して入力する
select_df = skill_df.query("スキル分類.str.match('{0}')".format(SkillClass.development))

# 表示する
select_df.drop(["スキルカテゴリ識別子", "スキル分類コード", "スキル分類識別子", "スキル項目識別子"], axis=1).head(500)

#### 1.2. スキル一覧（知識項目）を眺めてさらに対応しそうな知識項目を見つける。

In [ ]:
# データの結合
skill_know_df = pd.merge(skill_df, knowledge_df, on=["スキル項目コード"], how="inner")

# １で絞ったスキル項目に対応するスキル項目コードを入力し、知識項目を見つける
# 正規表現を使用可能。例：下二けた任意"S1100100.."
skill_item_code = "S130020050"
select_knowledge = skill_know_df.query("スキル項目コード.str.match('{0}')".format(skill_item_code))

# または、知識項目で検索する
#search_word = ".*アジャイル.*"
#select_knowledge = skill_know_df.query("知識項目.str.match('{0}')".format(search_word))

# 表示する
select_knowledge[["スキル項目コード", "知識項目コード", "スキル分類", "知識項目"]].head(500)

#### 1.3. Tags×スキル対応表に書き出す。
上記２で表示した、スキル項目コードと知識項目コードの中でTagsと適合するものを下記CSVファイルに追加する。
- <a href="./Notion/Tags×スキル対応表.csv">Tags×スキル対応表</a>

### 3. 習得したスキルの表示

In [40]:
# テーブルの結合
skill_know_df = pd.merge(skill_df, knowledge_df, on=["スキル項目コード"], how="inner")
pbl_skill_df = pd.merge(skill_know_df, tag_skill_df, on=["スキル項目コード", "知識項目コード"], how="right")
pbl_career_df = pd.merge(career_skill_df, pbl_skill_df, on=["スキル項目コード"], how="inner")
pbl_career_df = pd.merge(career_df, pbl_career_df, on=["専門分野コード"], how="inner")
pbl_career_df = pbl_career_df.drop(["スキル項目コード", "スキルカテゴリ識別子", "スキル分類コード", "スキル分類識別子", "スキル項目識別子", "知識項目コード", "知識項目識別子"], axis=1)
pbl_career_df

,専門分野コード,人材類型,人材像,出自,職種,専門分野,解説,スキルカテゴリ,スキル分類,スキル項目,知識項目,TAG_ID
0,HI-060-010,ソリューション系,テクニカルスペシャリスト,ITSS,ITスペシャリスト,プラットフォーム,ハードウェア、ソフトウェア関連の専門技術を活用し、顧客の環境に最適なシステム基盤の設計、構築...,テクノロジ,（システム） ソフトウェアの構築技術,開発ツール,バージョン管理システム,TAG-17
1,HI-060-020,ソリューション系,テクニカルスペシャリスト,ITSS,ITスペシャリスト,ネットワーク,ハードウェア、ソフトウェア関連の専門技術を活用し、顧客の環境に最適なシステム基盤の設計、構築...,テクノロジ,（システム） ソフトウェアの構築技術,開発ツール,バージョン管理システム,TAG-17
2,HI-060-030,ソリューション系,テクニカルスペシャリスト,ITSS,ITスペシャリスト,データベース,ハードウェア、ソフトウェア関連の専門技術を活用し、顧客の環境に最適なシステム基盤の設計、構築...,テクノロジ,（システム） ソフトウェアの構築技術,開発ツール,バージョン管理システム,TAG-17
3,HI-060-040,ソリューション系,テクニカルスペシャリスト,ITSS,ITスペシャリスト,アプリケーション共通基盤,ハードウェア、ソフトウェア関連の専門技術を活用し、顧客の環境に最適なシステム基盤の設計、構築...,テクノロジ,（システム） ソフトウェアの構築技術,開発ツール,バージョン管理システム,TAG-17
4,HI-060-050,ソリューション系,テクニカルスペシャリスト,ITSS,ITスペシャリスト,システム管理,ハードウェア、ソフトウェア関連の専門技術を活用し、顧客の環境に最適なシステム基盤の設計、構築...,テクノロジ,（システム） ソフトウェアの構築技術,開発ツール,バージョン管理システム,TAG-17
5,HI-060-060,ソリューション系,テクニカルスペシャリスト,ITSS,ITスペシャリスト,セキュリティ,ハードウェア、ソフトウェア関連の専門技術を活用し、顧客の環境に最適なシステム基盤の設計、構築...,テクノロジ,（システム） ソフトウェアの構築技術,開発ツール,バージョン管理システム,TAG-17
6,HI-070-010,ソリューション系,テクニカルスペシャリスト,ITSS,アプリケーションスペシャリスト,-,業種固有業務や汎用業務において、アプリケーション開発やパッケージ導入に関する専門技術を活用し...,メソドロジ,（利活用） サービスマネジメントプロセス,解決及び実現,バージョン管理,TAG-17
7,HI-070-010,ソリューション系,テクニカルスペシャリスト,ITSS,アプリケーションスペシャリスト,-,業種固有業務や汎用業務において、アプリケーション開発やパッケージ導入に関する専門技術を活用し...,メソドロジ,（利活用） サービスマネジメントプロセス,解決及び実現,バージョン管理ツール,TAG-17
8,HI-070-010,ソリューション系,テクニカルスペシャリスト,ITSS,アプリケーションスペシャリスト,-,業種固有業務や汎用業務において、アプリケーション開発やパッケージ導入に関する専門技術を活用し...,テクノロジ,（システム） ソフトウェアの基礎技術,プログラミング,バージョンとバージョン管理,TAG-17
9,HI-070-010,ソリューション系,テクニカルスペシャリスト,ITSS,アプリケーションスペシャリスト,-,業種固有業務や汎用業務において、アプリケーション開発やパッケージ導入に関する専門技術を活用し...,テクノロジ,（システム） ソフトウェアの構築技術,開発ツール,バージョン管理システム,TAG-17
